## Osher Dighorkar    
## Yuval Brunshtein   
## **Github Link**  : https://github.com/YuvalBru/Web_Scraping

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import datetime


## Getting Links And Data Extraction Functions


In [8]:
base_url = 'https://www.ad.co.il'


def get_all_renu_cars_linkss():
    renu_base_url = f"{base_url}/category/3?sp261=13894&pageindex=%page_num%"
    cars_urls = []

    current_page_number = 1

    while True:
        current_renu_page_url = renu_base_url.replace("%page_num%", str(current_page_number))
        response = requests.get(current_renu_page_url)

        soup = BeautifulSoup(response.content, 'html.parser')
        target_divs = soup.find_all('div', class_='card-body p-md-3')

        new_cars_urls = []

        for div in target_divs:
            a_tags = div.find('a', recursive=False)

            if a_tags is None:
                break

            partial_renu_car_url = a_tags.get("href")

            new_cars_urls.append(f"{base_url}{partial_renu_car_url}")

        if not new_cars_urls:
            break

        cars_urls.extend(new_cars_urls)
        current_page_number += 1

    return cars_urls

In [9]:
def get_data_from_renu_car_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    data_dict = {}
    table = soup.find('table', class_='table table-sm mb-4')
    desc = soup.find('meta', attrs={'name': 'description'})
    description_content = desc['content'] if desc else np.nan
    model_f = soup.find('h2', class_='card-title')
    model_ff = model_f.get_text(strip=True) if model_f else np.nan  
    price_ff = soup.find('div', class_ ="d-flex justify-content-between")
    if price_ff:
        price_f = price_ff.find_all('h2' , class_ = 'card-title')
        if len(price_f) == 2:
            fin_price = price_f[1].get_text(strip = True)
        else:
            fin_price = None
    imgs = soup.find('div' , class_ = 'row hide-on-mobile')
    if imgs:
        img_elements = imgs.find_all('img', class_='desktop-thumbnail bg-video') 
        data_indices = [int(img['data-index']) for img in img_elements]
        num_imgs = len(data_indices)
    else:
        num_imgs = 0
    manafacturer = re.findall('רנו',model_ff )
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        if len(columns) == 2:
            key = columns[0].get_text(strip=True)
            value = columns[1].get_text(strip=True)
            data_dict[key] = value
 
    return data_dict, description_content, model_ff, manafacturer[0] , fin_price , num_imgs

## Extracting Data And Sorting it Into A Data Set

In [10]:
def organize_url_translate(url):
    ht_content, ht_content['Description'], ht_content['Model'], ht_content['Manufactor'], ht_content['Price'], ht_content['Pic num'] = get_data_from_renu_car_url(url)
    hebrew_to_key = {
        'שנה': 'Year',
        'יד': 'Hand',
        'ת. הילוכים': 'Gear',
        'נפח': 'capacity_Engine',
        'סוג מנוע': 'Engine_type',
        'ק"מ': 'Km',
        'אזור': 'Area',
        'עיר': 'City' ,
        'Description' :'Description',
        'Model' : 'Model' ,
        'צבע' : 'Color' ,
        'תאריך יצירה' : 'Cre_date' ,
        'תאריך הקפצה' : 'Repub_date' ,
        'טסט עד' : 'Test' , 
        'מדד היצע' : 'Supply_score' , 
        'Pic num' : 'Pic num' ,
        'בעלות נוכחית' : 'Prev_ownership' ,
        'בעלות נוכחית' : 'Curr_ownership' , 
        'Price' : 'Price' , 
        'Manufactor' : 'Manufactor'
    }

    data2 = {
        'Year': None,
        'Hand': None,
        'Gear': None,
        'capacity_Engine': None,
        'Engine_type': None,
        'Km': None,
        'Area': None,
        'City': None,
        'Description': None,
        'Model': None,
        'Color': None,
        'Cre_date': None,
        'Repub_date': None,
        'Test': None,
        'Supply_score': None,
        'Pic num': None,
        'Prev_ownership': None,
        'Curr_ownership': None,
        'Price' : None,
        'Manufactor' : None
    }

    for hebrew_label, value in ht_content.items():
        if hebrew_label in hebrew_to_key:
            key = hebrew_to_key[hebrew_label]
            if key in ['Hand', 'Km', 'capacity_Engine','Supply_score' , 'Test'  , 'Year'] and value != None:
                if (key == 'Test'):
                    test_date = datetime.strptime(value, '%m/%Y')
                    current_date = datetime.now()
                    difference = test_date - current_date
                    days_between = difference.days
                    value = str(days_between)
                value = int(value.replace(',', ''))
            elif key == 'Price' and value != None:
                value = float(value.replace(',','').replace('₪' , '').replace(' ' , ''))
            data2[key] = value
    data_finale = pd.DataFrame([data2])
    return data_finale

In [11]:
renu_cars_in_between_years = []
min_year = 1980
max_year = 2015

renu_cars_links = get_all_renu_cars_linkss()

for car_link in renu_cars_links:
    data = get_data_from_renu_car_url(car_link)
    year = int(data[0]['שנה'])
    if min_year <= year <= max_year:
        renu_cars_in_between_years.append(car_link)

In [12]:
dict_for_df = {
        'Year': None,
        'Hand': None,
        'Gear': None,
        'capacity_Engine': None,
        'Engine_type': None,
        'Km': None,
        'Area': None,
        'City': None,
        'Description': None,
        'Model': None,
        'Color': None,
        'Cre_date': None,
        'Repub_date': None,
        'Test': None,
        'Supply_score': None,
        'Pic num': None,
        'Prev_ownership': None,
        'Curr_ownership': None,
        'Price' : None,
        'Manufactor' : None
    }
df = pd.DataFrame(columns=dict_for_df.keys())
for item in renu_cars_in_between_years:
    df = pd.concat([df , organize_url_translate(item)], ignore_index = True)
df

,Year,Hand,Gear,capacity_Engine,Engine_type,Km,Area,City,Description,Model,Color,Cre_date,Repub_date,Test,Supply_score,Pic num,Prev_ownership,Curr_ownership,Price,Manufactor
0,2013,1,אוטומטית,1200,חשמלי,70000,מושבים בשרון,שער אפרים,פלואנס חשמלי בטר פלייס\nיש לחדש סוללה וטסט,רנו פלואנס חשמלי,כחול כהה מטאלי,None,None,None,None,4,None,פרטית,25000.0,רנו
1,2008,3,אוטומטית,1600,דיזל,267000,באר שבע והסביבה,באר שבע,טסט עד 1.7 מטופלת,רנו מגאן II,כחול כהה,None,None,None,None,2,None,פרטית,5000.0,רנו
2,2013,3,אוטומטית,1500,דיזל,290000,פרדס חנה - כרכור,חריש,*רכב במצב מעולה *שמור ומוטפל בזמן\n*חסכוני בדל...,רנו מגאן,אפור מטאלי,None,None,240,None,2,None,פרטית,1.0,רנו
3,2014,2,אוטומטית,6000,בנזין,58,ירושלים והסביבה,ירושלים,רכב פצצה במצב מצויין \nחסכוני בטרוף !! שמור מא...,רנו קליאו,אדום,None,None,56,None,5,None,פרטית,30000.0,רנו
4,2008,3,רובוטית,2000,בנזין,260000,None,שילה,"רכב טוב מאוד, אצלי כבר 6 שנים ולא אכזב, מהשבע-...",רנו גרנד סניק,אפור מטאלי,None,None,None,None,4,None,פרטית,7500.0,רנו
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2013,1,ידנית,1150,בנזין,None,None,ידידיה,את הרכב קניתי ישירות מהיבואן.,רנו קליאו,None,None,None,None,None,0,None,None,18000.0,רנו
66,2012,2,אוטומטית,90,חשמלי,42300,בית שמש והסביבה,בית שמש,"רכב חשמלי 100%\r\nשקט אמין נוסע 80 ק"" ללא דלק ...",רנו פלואנס חשמלי,None,None,None,None,None,0,None,None,27000.0,רנו
67,2015,3,אוטומטית,1197,בנזין,85000,ירושלים והסביבה,מבשרת ציון,"קפצור גרסה מעוצבת, בצבע אדום גג לבן. נמכר עקב ...",רנו קפצ`ור,None,None,None,None,None,0,None,None,44500.0,רנו
68,2014,2,אוטומטית,1461,טורבו דיזל,None,תל אביב,תל אביב יפו,NaN,רנו 25,None,None,None,None,None,0,None,None,28000.0,רנו


## Changing Columns to Categorical and Date Types

In [13]:
categories = ["ציבורית", "ליסינג","פרטית"]
df['Curr_ownership'] = pd.Categorical(df['Curr_ownership'], categories=categories)

categories = ["דיזל", "חשמלי","גז","היבריד","בנזין"]
df['Engine_type'] = pd.Categorical(df['Engine_type'], categories=categories)

categories = ["פרטית","ליסינג","ציבורית"]
df['Prev_ownership'] = pd.Categorical(df['Prev_ownership'], categories=categories)

categories = ["רובוטית","ידנית","אוטומטית"]
df['Gear'] = pd.Categorical(df['Gear'], categories=categories)


In [14]:

df['Cre_date'] = pd.to_datetime(df['Cre_date'])
df['Repub_date'] = pd.to_datetime(df['Repub_date'])
df.replace([pd.NaT, None], np.nan, inplace=True)


In [15]:
df

,Year,Hand,Gear,capacity_Engine,Engine_type,Km,Area,City,Description,Model,Color,Cre_date,Repub_date,Test,Supply_score,Pic num,Prev_ownership,Curr_ownership,Price,Manufactor
0,2013,1,אוטומטית,1200,חשמלי,70000.0,מושבים בשרון,שער אפרים,פלואנס חשמלי בטר פלייס\nיש לחדש סוללה וטסט,רנו פלואנס חשמלי,כחול כהה מטאלי,NaT,NaT,NaN,NaN,4,NaN,פרטית,25000.0,רנו
1,2008,3,אוטומטית,1600,דיזל,267000.0,באר שבע והסביבה,באר שבע,טסט עד 1.7 מטופלת,רנו מגאן II,כחול כהה,NaT,NaT,NaN,NaN,2,NaN,פרטית,5000.0,רנו
2,2013,3,אוטומטית,1500,דיזל,290000.0,פרדס חנה - כרכור,חריש,*רכב במצב מעולה *שמור ומוטפל בזמן\n*חסכוני בדל...,רנו מגאן,אפור מטאלי,NaT,NaT,240.0,NaN,2,NaN,פרטית,1.0,רנו
3,2014,2,אוטומטית,6000,בנזין,58.0,ירושלים והסביבה,ירושלים,רכב פצצה במצב מצויין \nחסכוני בטרוף !! שמור מא...,רנו קליאו,אדום,NaT,NaT,56.0,NaN,5,NaN,פרטית,30000.0,רנו
4,2008,3,רובוטית,2000,בנזין,260000.0,NaN,שילה,"רכב טוב מאוד, אצלי כבר 6 שנים ולא אכזב, מהשבע-...",רנו גרנד סניק,אפור מטאלי,NaT,NaT,NaN,NaN,4,NaN,פרטית,7500.0,רנו
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2013,1,ידנית,1150,בנזין,NaN,NaN,ידידיה,את הרכב קניתי ישירות מהיבואן.,רנו קליאו,NaN,NaT,NaT,NaN,NaN,0,NaN,NaN,18000.0,רנו
66,2012,2,אוטומטית,90,חשמלי,42300.0,בית שמש והסביבה,בית שמש,"רכב חשמלי 100%\r\nשקט אמין נוסע 80 ק"" ללא דלק ...",רנו פלואנס חשמלי,NaN,NaT,NaT,NaN,NaN,0,NaN,NaN,27000.0,רנו
67,2015,3,אוטומטית,1197,בנזין,85000.0,ירושלים והסביבה,מבשרת ציון,"קפצור גרסה מעוצבת, בצבע אדום גג לבן. נמכר עקב ...",רנו קפצ`ור,NaN,NaT,NaT,NaN,NaN,0,NaN,NaN,44500.0,רנו
68,2014,2,אוטומטית,1461,NaN,NaN,תל אביב,תל אביב יפו,NaN,רנו 25,NaN,NaT,NaT,NaN,NaN,0,NaN,NaN,28000.0,רנו
